<a href="https://colab.research.google.com/github/jdowner212/cs577_addernet/blob/main/AdderNet_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from data import load_cifar_data
from model import Model
import layers

In [4]:
data_dir = f'{root}/cifar-10-batches-py'
X_trn_c10, y_trn_c10, X_tst_c10, y_tst_c10, X_val_c10, y_val_c10 = load_cifar_data(data_dir,tiny=True)

In [5]:
this_model = Model(loss_name='cat_cross_entropy')

this_model.add(layers.adder_layer(output_channels=8,kernel_size=3,stride=1,padding=1,adaptive_eta=0.1))
this_model.add(layers.Activation('relu'))
this_model.add(layers.MaxPool(pool_size=2))
this_model.add(layers.batch_norm_layer())


this_model.add(layers.Flatten())
this_model.add(layers.FullyConnected(output_channels=64))
this_model.add(layers.Activation('relu'))
this_model.add(layers.FullyConnected(output_channels=10))
this_model.add(layers.Activation('softmax'))

In [6]:
history = this_model.fit(X_trn_c10,y_trn_c10,epochs=20,batch_size=16,learning_rate=1e-05,x_val=X_val_c10,y_val=y_val_c10)

0


2022-11-16 19:19:16.779092: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-11-16 19:19:16.779126: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: quinlan-bock
2022-11-16 19:19:16.779131: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: quinlan-bock
2022-11-16 19:19:16.779277: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.76.0
2022-11-16 19:19:16.779291: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.76.0
2022-11-16 19:19:16.779294: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 515.76.0
2022-11-16 19:19:16.779684: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-cri

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(range(20),history['accuracy'],label='train')
plt.plot(range(20),history['val_accuracy'],label='val')
plt.legend()
plt.xlabel('Epochs')
plt.xticks(range(0,21,2))
plt.ylim(0,0.3)
plt.title('Accuracy -- our AdderNet implementation')
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(range(20),history['loss'],label='train')
plt.plot(range(20),history['val_loss'],label='val')
plt.legend()
plt.xlabel('Epochs')
plt.xticks(range(0,21,2))
plt.ylim(0,0.1)
plt.title('Loss -- our AdderNet implementation')
plt.show()



In [ ]:
#this_model = Model(loss_name='cat_cross_entropy')
# l1 = X_trn_c10
# l1=adder_layer(output_channels=8,kernel_size=3,stride=1,padding=1).forward(l1)
# print(l1.shape)
# l2=Activation('relu').forward(l1)
# print(l2.shape)
# l3=MaxPool(pool_size=2).forward(l2)
# print(l3.shape)
# l4=batch_norm_layer().forward(l3)
# print(l4.shape)

# l5=Flatten().forward(l4)
# print(l5.shape)
# l6=FullyConnected(output_channels=64).forward(l5)
# print(l6.shape)
# l7=Activation('relu').forward(l6)
# print(l7.shape)
# l8=FullyConnected(output_channels=10).forward(l7)
# print(l8.shape)
# l9=Activation('softmax').forward(l8)
# print(l9.shape)

In [ ]:
cnn = Model(loss_name='cat_cross_entropy')

cnn.add(conv_layer(output_channels=8,kernel_size=3,stride=1,padding=1))
cnn.add(Activation('relu'))
cnn.add(MaxPool(pool_size=2))
cnn.add(batch_norm_layer())


cnn.add(Flatten())

cnn.add(FullyConnected(output_channels=64))
cnn.add(batch_norm_layer())
cnn.add(Activation('relu'))
cnn.add(FullyConnected(output_channels=10))
cnn.add(batch_norm_layer())
cnn.add(Activation('softmax'))

NameError: ignored

In [ ]:
cnn.fit(X_trn_c10,y_trn_c10,10,1e-05,X_val_c10,y_val_c10)

0
0


ValueError: ignored

In [ ]:
def add2d(X, K):  
    """Compute 2D cross-correlation."""
    h, w = K.shape
    Y = tf.Variable(tf.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1)))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j].assign(tf.reduce_sum(
                X[i: i + h, j: j + w] + K))
    return Y


class adder_2d(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()

    def build(self, kernel_size):
        initializer = tf.random_normal_initializer()
        self.weight = self.add_weight(name='w', shape=kernel_size,
                                      initializer=initializer)
        self.bias = self.add_weight(name='b', shape=(1, ),
                                    initializer=initializer)

    def call(self, inputs):
        return add2d(inputs, self.weight) + self.bias


class add_it(tf.keras.layers.Conv2D):
    def convolution_op(self, inputs, kernel):
        mean, var = tf.nn.moments(kernel, axes=[0, 1, 2], keepdims=True)
        return tf.nn.conv2d(
            inputs,
            (kernel - mean) / tf.sqrt(var + 1e-10),
            padding="VALID",
            strides=list(self.strides),
            name=self.__class__.__name__,
        )

In [ ]:
def conv_single_step(window, filter_, bias):
    """
    window -- k_h x k_w x k_d
    filter_ -- k_h x k_w x k_d
    b      -- 1x1x1
    Z      -- scalar
    """
    out = np.sum((np.multiply(window,filter_) + bias.astype(float))).astype(float)
    
    return out

class conv_layer(Layer):
    def __init__(self,output_channels,kernel_size=3,stride=1,padding=0):#,similarity_f = L1):
        self.output_channels = output_channels


        self.output_channels = output_channels
        self.adaptive_eta=0

        self.kernel_size=kernel_size        
        self.stride = stride
        self.padding = padding



    def forward(self,X):
        """    
        X       -- n_tensors x H x W x c_in
        filters -- c_out x k_H x k_W x c_in
        b       -- c_out x 1 x 1 x 1
        Z       -- n_tensors x H_new x W_new, c_out
        cache   -- info needed for backward pass
        """
        self.input = X

        # in case input size not given
        self.input_channels = X.shape[-1]

        self.filters = np.random.normal(loc=0,scale=1,size=(self.output_channels, self.kernel_size, self.kernel_size, self.input_channels))
        self.bias    = np.random.normal(loc=0,scale=1,size=(self.output_channels, 1,1 ,self.input_channels))
        
        filters,stride,padding,bias = self.filters, self.stride, self.padding, self.bias
        n_tensors, H,   W,   c_in = X.shape
        c_out,     H_k, W_k, c_in = filters.shape
        n_filters = c_out

        X_padded = np.pad(X, ((0,0), (padding,padding), (padding,padding), (0,0)), 'constant', constant_values = (0,0))
        H_new = int((H + 2*padding - H_k)/stride)+1
        W_new = int((W + 2*padding - W_k)/stride)+1

        Z = np.zeros([n_tensors, H_new, W_new, c_out])

        for i in range(n_tensors):           # traverse batch
            this_img = X_padded[i,:,:,:]     # select ith image in batch
            for f in range(n_filters):       # traverse filters
                this_filter = filters[f,:,:,:]
                this_bias   = bias[f,:,:,:]
                for h in range(H_new):       # traverse height
                    for w in range(W_new):   # traverse width
                        v0,v1 = h*stride, h*stride + H_k
                        h0,h1 = w*stride, w*stride + W_k
                        this_window = this_img[v0:v1,h0:h1,:]

                        Z[i, h, w, f] = conv_single_step(this_window, this_filter, this_bias) 

        assert Z.shape == (n_tensors, H_new, W_new, n_filters)

        self.output = Z
        self.cache = X, filters, bias, stride, padding
        
        return self.output

    def backward(self, upstream_g, learning_rate):
        """
        upstream_g (dL/dZ) -- n_tensors x H_up x W_up x c_up
        cache (values from previous layers) -- (X, W, B, s, p)               
        
        Output:
        dX -- dL/dX, shape n_tensors x H_down x W_down x c_down
        dF -- dL/dW, shape n_filters x k x k x k
        dB -- dL/dB, shape n_filters x 1 x 1 x 1
        """
        X, filters, bias, stride, padding = self.cache

        n_tensors, H_down, W_down, c_down = X.shape
        n_filters, H_k,    W_k,    c_down = filters.shape
        n_tensors, H_up,   W_up,   c_up   = upstream_g.shape
        
        dX       = np.zeros_like(X)                           
        dfilters = np.zeros_like(filters)
        dbias    = np.zeros_like(bias)

        X_padded  = np.pad(X,  ((0,0), (padding,padding), (padding,padding), (0,0)), 'constant', constant_values = (0,0))
        dX_padded = np.pad(dX, ((0,0), (padding,padding), (padding,padding), (0,0)), 'constant', constant_values = (0,0))
        
        for i in range(n_tensors):                       
            x = X_padded[i]
            dx = dX_padded[i]
            
            for h in range(H_up):                   # traverse height
                for w in range(W_up):               # traverse width
                    for c in range(c_up):           # traverse filters
                        
                        v0,v1 = h,h+H_k
                        h0,h1 = w,w+W_k
                        
                        x_window = x[v0:v1, h0:h1, :]
                        f_window = filters[c,:,:,:]

                        dx_local = hard_tanh(f_window-x_window)
                        df_local = x_window-f_window

                        g = upstream_g[i, h, w, c]

                        dx[v0:v1, v0:v1, :] += np.multiply(dx_local,g)
                        dfilters[c,:,:,:]   += np.multiply(df_local,g)
                        dbias[c,:,:,:]      += g
                        
            dX[i, :, :, :] = dx[padding:-padding, padding:-padding, :]
        
        assert(dX.shape == (n_tensors, H_down, W_down, c_down))

        self.filters -= learning_rate*dfilters
        self.bias    -= learning_rate*dbias

        return dX